In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, input_nodes, hidden_1_nodes, hidden_2_nodes, output_nodes, learning_rate):
        
        self.name = gate_name
                
        # 2층 hidden1 layer 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 hidden2 layer 
        self.W3 = np.random.rand(hidden_1_nodes, hidden_2_nodes)
        self.b3 = np.random.rand(hidden_2_nodes)
                        
        # 4층 output layer 
        self.W4 = np.random.rand(hidden_2_nodes, output_nodes)
        self.b4 = np.random.rand(output_nodes)
        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
            
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z1 = np.dot(input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

In [11]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 2  # hidden 1 nodes
h2_nodes = 2  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    and_obj.train(and_xdata.reshape(4,2), and_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", and_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

AND object is created
step =  0 loss value =  4.979448734123029
step =  500 loss value =  2.2034705307682882
step =  1000 loss value =  0.38551151668714684
step =  1500 loss value =  0.0632385005875218
step =  2000 loss value =  0.03217032425684685
step =  2500 loss value =  0.02133653341270391
step =  3000 loss value =  0.015899977675774703
step =  3500 loss value =  0.012648399745778558
step =  4000 loss value =  0.010490438015429811
step =  4500 loss value =  0.008956011258476325
step =  5000 loss value =  0.007810028675496215
step =  5500 loss value =  0.006922105157690989
step =  6000 loss value =  0.006214202751851174
step =  6500 loss value =  0.005636797697183128
step =  7000 loss value =  0.0051569669172846526
step =  7500 loss value =  0.004751980322937711
step =  8000 loss value =  0.004405645159054016
step =  8500 loss value =  0.004106116684552167
step =  9000 loss value =  0.003844532547504223
step =  9500 loss value =  0.0036141309239514346
step =  10000 loss value =  0.

In [12]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [13]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 2  # hidden 1 nodes
h2_nodes = 2  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    or_obj.train(or_xdata.reshape(4,2), or_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", or_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

OR object is created
step =  0 loss value =  2.2450606257733576
step =  500 loss value =  0.9190790658901703
step =  1000 loss value =  0.07373894022306908
step =  1500 loss value =  0.034194233981636776
step =  2000 loss value =  0.02201306753454251
step =  2500 loss value =  0.016178311544004044
step =  3000 loss value =  0.012771013498211662
step =  3500 loss value =  0.010541836685130802
step =  4000 loss value =  0.008971586083448465
step =  4500 loss value =  0.00780650771310443
step =  5000 loss value =  0.0069080927791725505
step =  5500 loss value =  0.006194403321550634
step =  6000 loss value =  0.00561390014848096
step =  6500 loss value =  0.005132557630412948
step =  7000 loss value =  0.004727014893400954
step =  7500 loss value =  0.004380705258081845
step =  8000 loss value =  0.004081556010311152
step =  8500 loss value =  0.0038205627394406784
step =  9000 loss value =  0.003590873647514187
step =  9500 loss value =  0.0033871830976312297
step =  10000 loss value =  

In [14]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [7]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 2  # hidden 1 nodes
h2_nodes = 2  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    nand_obj.train(nand_xdata.reshape(4,2), nand_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", nand_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

NAND object is created
step =  0 loss value =  2.461776172899573
step =  500 loss value =  2.239635549489359
step =  1000 loss value =  1.2278511148607645
step =  1500 loss value =  0.10920383926733795
step =  2000 loss value =  0.048390093653032595
step =  2500 loss value =  0.030379884998929532
step =  3000 loss value =  0.021929555604928305
step =  3500 loss value =  0.017066808952965185
step =  4000 loss value =  0.013922647576966247
step =  4500 loss value =  0.011729879155697467
step =  5000 loss value =  0.010117078019971509
step =  5500 loss value =  0.008883094829656369
step =  6000 loss value =  0.007909746775075537
step =  6500 loss value =  0.007123150386054195
step =  7000 loss value =  0.0064748001626017595
step =  7500 loss value =  0.0059315651331987105
step =  8000 loss value =  0.00547006271318614
step =  8500 loss value =  0.005073329936163823
step =  9000 loss value =  0.004728768785143972
step =  9500 loss value =  0.0044268315771159176
step =  10000 loss value =  

In [8]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [9]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 2  # hidden 1 nodes
h2_nodes = 2  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    xor_obj.train(xor_xdata.reshape(4,2), xor_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", xor_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

XOR object is created
step =  0 loss value =  2.8674512037801634
step =  500 loss value =  2.772129289277429
step =  1000 loss value =  2.7714758113303377
step =  1500 loss value =  2.770253598432928
step =  2000 loss value =  2.76723269704859
step =  2500 loss value =  2.7553039240373156
step =  3000 loss value =  2.571199851069709
step =  3500 loss value =  1.9403262309293632
step =  4000 loss value =  0.13173933009832203
step =  4500 loss value =  0.05097347400933353
step =  5000 loss value =  0.03078658464526788
step =  5500 loss value =  0.021798697085576177
step =  6000 loss value =  0.016759123342817157
step =  6500 loss value =  0.01355159811603142
step =  7000 loss value =  0.011338617842630573
step =  7500 loss value =  0.009723868952272407
step =  8000 loss value =  0.0084960719265552
step =  8500 loss value =  0.0075325164006330055
step =  9000 loss value =  0.006757165522720442
step =  9500 loss value =  0.006120450778072496
step =  10000 loss value =  0.005588709695961582

In [10]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = xor_obj.accuracy2(test_data)

XOR 

Accuracy =>  1.0
Accuracy =>  1.0
